# Agent

Dans ce *notebook*, **nous allons construire un agent simple en utilisant LangGraph**.

Ce notebook fait parti du cours <a href="https://huggingface.co/learn/agents-course/fr">sur les agents d'Hugging Face</a>, un cours gratuit qui vous guidera, du **niveau débutant à expert**, pour comprendre, utiliser et construire des agents.
![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Comme nous l'avons vu dans l'Unité 1, un agent a besoin de 3 étapes telles qu'introduites dans l'architecture ReAct :
[ReAct](https://react-lm.github.io/), une architecture générale d'agent.

* `act` - laisser le modèle appeler des outils spécifiques
* `observe` - transmettre la sortie de l'outil au modèle
* `reason` - permet au modèle de raisonner sur la sortie de l'outil pour décider de ce qu'il doit faire ensuite (par exemple, appeler un autre outil ou simplement répondre directement).

![Agent](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/LangGraph/Agent.png)

In [ ]:
%pip install -q -U langchain_openai langchain_core langgraph

In [ ]:
import os

# Veuillez configurer votre propre clé
os.environ["OPENAI_API_KEY"] = "sk-xxxxxx"

In [ ]:
import base64
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

vision_llm = ChatOpenAI(model="gpt-4o")


def extract_text(img_path: str) -> str:
    """
    Extract text from an image file using a multimodal model.

    Args:
        img_path: A local image file path (strings).

    Returns:
        A single string containing the concatenated text extracted from each image.
    """
    all_text = ""
    try:

        # Lire l'image et l'encoder en base64
        with open(img_path, "rb") as image_file:
            image_bytes = image_file.read()

        image_base64 = base64.b64encode(image_bytes).decode("utf-8")

        # Préparer le prompt en incluant les données de l'image base64
        message = [
            HumanMessage(
                content=[
                    {
                        "type": "text",
                        "text": (
                            "Extract all the text from this image. "
                            "Return only the extracted text, no explanations."
                        ),
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{image_base64}"
                        },
                    },
                ]
            )
        ]

        # Appeler le VLM
        response = vision_llm.invoke(message)

        # Ajouter le texte extrait
        all_text += response.content + "\n\n"

        return all_text.strip()
    except Exception as e:
        # Vous pouvez choisir de renvoyer une chaîne vide ou un message d'erreur.
        error_msg = f"Error extracting text: {str(e)}"
        print(error_msg)
        return ""


llm = ChatOpenAI(model="gpt-4o")


def divide(a: int, b: int) -> float:
    """Divide a and b."""
    return a / b


tools = [
    divide,
    extract_text
]
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

Créons notre LLM et demandons-lui le comportement global souhaité de l'agent.

In [ ]:
from typing import TypedDict, Annotated, Optional
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # Le document d'entrée
    input_file: Optional[str]  # Contient le chemin d'accès au fichier, le type (PNG)
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.utils.function_calling import convert_to_openai_tool


def assistant(state: AgentState):
    # Message système
    textual_description_of_tool = """
extract_text(img_path: str) -> str:
    Extract text from an image file using a multimodal model.

    Args:
        img_path: A local image file path (strings).

    Returns:
        A single string containing the concatenated text extracted from each image.
divide(a: int, b: int) -> float:
    Divide a and b
"""
    image = state["input_file"]
    sys_msg = SystemMessage(content=f"You are an helpful agent that can analyse some images and run some computatio without provided tools :\n{textual_description_of_tool} \n You have access to some otpional images. Currently the loaded images is : {image}")

    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])], "input_file": state["input_file"]}

Nous définissons un nœud `tools` avec notre liste d'outils.

Le noeud `assistant` est juste notre modèle avec les outils liés.

Nous créons un graphe avec les noeuds `assistant` et `tools`.

Nous ajoutons l'arête `tools_condition`, qui route vers `End` ou vers `tools` selon que le `assistant` appelle ou non un outil.

Maintenant, nous ajoutons une nouvelle étape :

Nous connectons le noeud `tools` au `assistant`, formant une boucle.

* Après l'exécution du noeud `assistant`, `tools_condition` vérifie si la sortie du modèle est un appel d'outil.
* Si c'est le cas, le flux est dirigé vers le noeud `tools`.
* Le noeud `tools` se connecte à `assistant`.
* Cette boucle continue tant que le modèle décide d'appeler des outils.
* Si la réponse du modèle n'est pas un appel d'outil, le flux est dirigé vers END, mettant fin au processus.

In [ ]:
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

# Graphe
builder = StateGraph(AgentState)

# Définir les nœuds : ce sont eux qui font le travail
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Définir les arêtes : elles déterminent la manière dont le flux de contrôle se déplace
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # Si le dernier message (résultat) de l'assistant est un appel d'outil -> tools_condition va vers tools
    # Si le dernier message (résultat) de l'assistant n'est pas un appel d'outil -> tools_condition va à END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()

# Afficher
display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
messages = [HumanMessage(content="Divide 6790 by 5")]

messages = react_graph.invoke({"messages": messages, "input_file": None})

In [ ]:
for m in messages['messages']:
    m.pretty_print()

## Programme d'entraînement
M. Wayne a laissé une note avec son programme d'entraînement pour la semaine. J'ai trouvé une recette pour le dîner, laissée dans une note.

Vous pouvez trouver le document [ICI](https://huggingface.co/datasets/agents-course/course-images/blob/main/en/unit2/LangGraph/Batman_training_and_meals.png), alors téléchargez-le et mettez-le dans le dossier local.

![Training](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/LangGraph/Batman_training_and_meals.png)

In [ ]:
messages = [HumanMessage(content="According the note provided by MR wayne in the provided images. What's the list of items I should buy for the dinner menu ?")]

messages = react_graph.invoke({"messages": messages, "input_file": "Batman_training_and_meals.png"})

In [ ]:
for m in messages['messages']:
    m.pretty_print()